In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  R, C = map(int, input().split())
  G = [list(map(lambda x: x - 48, input().rstrip())) for _ in range(R)]
  
  def measure(pos) : #pos 칸으로부터 최대로 만들 수 있는 크기
    size = 1
    x, y = pos
    while True :
      for dx, dy in (1, -1), :
        nx, ny = x + dx * size, y + dy * size
        if not (0 <= nx < R and 0 <= ny < C): return size
        if not G[nx][ny]: return size
        size += 1

  def check(size, pos) :
    x, y = pos
    for dx, dy in (1, -1), (1, 1), (-1, 1), (-1, -1) :
      for sc in range(size) :
        nx, ny = x + dx, y + dy
        if not (0 <= nx < R and 0 <= ny < C) : return False
        if not G[nx][ny] : return False
        x, y = nx, ny
    return True
  
  ans = 0
  true_m_size = min(R, C) // 2
  for i, l in enumerate(G) :
    for j, v in enumerate(l) :
      if not v : continue
      if ans == true_m_size : break
      m_size = min(measure((i, j)) + 1, true_m_size)
      for size in range(ans - 1, m_size) :
        if check(size, (i, j)) : ans = size + 1
        else: break
  print(ans)

sol()

- 반례: 
5 5
11111
11111
11011
11111
11111


In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  R, C = map(int, input().split())
  G = [list(map(lambda x: x - 48, input().rstrip())) for _ in range(R)]
  
  def measure(pos) : #pos 칸으로부터 최대로 만들 수 있는 크기
    size = 1
    x, y = pos
    while True :
      for dx, dy in (1, -1), :
        nx, ny = x + dx * size, y + dy * size
        if not (0 <= nx < R and 0 <= ny < C): return size
        if not G[nx][ny]: return size
        size += 1

  def check(size, pos) :
    x, y = pos
    for dx, dy in (1, -1), (1, 1), (-1, 1), (-1, -1) :
      for _ in range(size) :
        nx, ny = x + dx, y + dy
        if not (0 <= nx < R and 0 <= ny < C) : return False
        if not G[nx][ny] : return False
        x, y = nx, ny
    return True
  
  ans = 0
  true_m_size = (min(R, C) + 1) // 2 #최적해의 upper bound
  for i, l in enumerate(G) :
    for j, v in enumerate(l) :
      if not v : continue
      if ans == true_m_size : break
      m_size = min(measure((i, j)) + 1, true_m_size)
      for size in reversed(range(ans - 1, m_size)) :
        if check(size, (i, j)) : ans = max(ans, size + 1)
  print(ans)

sol()

- AC를 맞았는데, 시간복잡도가 $O(N^4)$ 이다. 불가능해야 하는데 정답인걸 봐선 데이터셋에 문제가 있는듯 하다. 틀왜맞이죠?

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  R, C = map(int, input().split())
  G = [[0] + list(map(lambda x: x - 48, input().rstrip())) + [0] for _ in range(R)]
  G = [[0] * (C+2)] + G + [[0] * (C+2)] 
  ld = [[0] * (C+2) for _ in range(R+2)] #해당 index에서 왼하단 방향으로 연속된 1의 개수
  rd = [[0] * (C+2) for _ in range(R+2)]
  lu = [[0] * (C+2) for _ in range(R+2)]
  ru = [[0] * (C+2) for _ in range(R+2)]
  
  for i in range(R, 0, -1) :
    for j in range(1, C+1) :
      if not G[i][j] : continue
      ld[i][j] = ld[i+1][j-1]+1
      rd[i][j] = rd[i+1][j+1]+1
  
  for i in range(1, R+1) :
    for j in range(1, C+1) :
      if not G[i][j] : continue
      lu[i][j] = lu[i-1][j-1]+1
      ru[i][j] = ru[i-1][j+1]+1
  
  ans = 0
  for i in range(1, R+1) :
    for j in range(1, C+1) :
      for k in range(not not ans, min(ld[i][j], rd[i][j]) + 1) :
        x = i+2*(k-1)
        if x > R+1 : break
        if G[x][j] and lu[x][j] >= k and ru[x][j] >= k :
          ans = max(ans, k)
      for k in range(not not ans, min(ru[i][j], rd[i][j]) + 1) :
        y = j+2*(k-1)
        if y > C+1 : break
        if G[i][y] and lu[i][y] >= k and ld[i][y] >= k :
          ans = max(ans, k)
  print(ans)
sol()

5 5
01100
01011
11111
01111
11111
;;
5 5
01100
00011
11111
01111
11111
;;
4 4
0000
0000
0000
0000
;;
3 6
111000
101111
111111
;;
5 5
11111
11111
11011
11111
11111

### 풀이
- 그래프를 순회하면서 각 대각선의 방향대로 연속하는 1의 개수를 저장한다.